library

In [5]:
!pip install huggingface_hub ultralytics supervision pillow

  Using cached supervision-0.27.0-py3-none-any.whl.metadata (13 kB)


test-model

In [ ]:
# load libraries
from huggingface_hub import hf_hub_download
from ultralytics import YOLO
from supervision import Detections, BoxAnnotator
from PIL import Image
import os
import numpy as np
import shutil

# สร้างโฟลเดอร์สำหรับเก็บโมเดล
models_folder = "/Users/nene/pestguard_IoTHackathon_sit/ai/models"
os.makedirs(models_folder, exist_ok=True)

# download model จาก Hugging Face
model_filename = "yolov8_face_detection.pt"
local_model_path = os.path.join(models_folder, model_filename)

# ถ้ายังไม่มีโมเดลในโฟลเดอร์ ให้ดาวน์โหลดและคัดลอกมา
if not os.path.exists(local_model_path):
    print("กำลังดาวน์โหลดโมเดลจาก Hugging Face...")
    downloaded_model_path = hf_hub_download(repo_id="arnabdhar/YOLOv8-Face-Detection", filename="model.pt")
    shutil.copy2(downloaded_model_path, local_model_path)
    print(f"บันทึกโมเดลที่: {local_model_path}")
else:
    print(f"ใช้โมเดลที่มีอยู่แล้วที่: {local_model_path}")

# load model
model = YOLO(local_model_path)
print(f"โมเดลพร้อมใช้งาน: {local_model_path}")

# สร้างโฟลเดอร์สำหรับเก็บผลลัพธ์
output_folder = "/Users/nene/pestguard_IoTHackathon_sit/ai/result"
os.makedirs(output_folder, exist_ok=True)

# inference
image_path = "/Users/nene/pestguard_IoTHackathon_sit/imagetrain/IMG_1924.png"
image = Image.open(image_path)
output = model(image)
results = Detections.from_ultralytics(output[0])

# วาด bounding boxes บนภาพ
box_annotator = BoxAnnotator()
annotated_image = box_annotator.annotate(
    scene=np.array(image),
    detections=results
)

# บันทึกภาพที่วาด bounding boxes แล้ว
output_image_path = os.path.join(output_folder, os.path.basename(image_path))
Image.fromarray(annotated_image).save(output_image_path)
print(f"บันทึกภาพผลลัพธ์ที่: {output_image_path}")
print(f"พบ {len(results)} ใบหน้า")



ใช้โมเดลที่มีอยู่แล้วที่: /Users/nene/pestguard_IoTHackathon_sit/ai/models/yolov8_face_detection.pt
โมเดลพร้อมใช้งาน: /Users/nene/pestguard_IoTHackathon_sit/ai/models/yolov8_face_detection.pt

0: 640x480 2 FACEs, 97.7ms
Speed: 6.5ms preprocess, 97.7ms inference, 5.7ms postprocess per image at shape (1, 3, 640, 480)
บันทึกภาพผลลัพธ์ที่: /Users/nene/pestguard_IoTHackathon_sit/ai/result/IMG_1924.png
พบ 2 ใบหน้า


crop-image

In [30]:
from ultralytics import YOLO
import cv2
import os

def crop_from_bbox(image, bbox):
    h, w = image.shape[:2]
    x1, y1, x2, y2 = bbox

    x1 = max(0, int(x1))
    y1 = max(0, int(y1))
    x2 = min(w, int(x2))
    y2 = min(h, int(y2))

    return image[y1:y2, x1:x2]

model = YOLO("/Users/nene/pestguard_IoTHackathon_sit/ai/models/yolov8_face_detection.pt")

input_folder = "/Users/nene/pestguard_IoTHackathon_sit/imagetrain"
output_folder = "/Users/nene/pestguard_IoTHackathon_sit/cropped_faces"

os.makedirs(output_folder, exist_ok=True)

img_count = 0

for img_name in os.listdir(input_folder):
    if not img_name.lower().endswith((".jpg", ".jpeg", ".png")):
        continue

    img_path = os.path.join(input_folder, img_name)
    img = cv2.imread(img_path)

    if img is None:
        continue

    results = model(img)

    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = box.tolist()
        face = crop_from_bbox(img, (x1, y1, x2, y2))

        if face.size == 0:
            continue

        save_path = os.path.join(
            output_folder,
            f"{os.path.splitext(img_name)[0]}_face_{img_count}.png"
        )

        cv2.imwrite(save_path, face)
        img_count += 1



0: 640x480 1 FACE, 41.8ms
Speed: 1.6ms preprocess, 41.8ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 32.9ms
Speed: 1.9ms preprocess, 32.9ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 38.6ms
Speed: 2.4ms preprocess, 38.6ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 40.4ms
Speed: 1.7ms preprocess, 40.4ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 34.4ms
Speed: 1.8ms preprocess, 34.4ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 38.2ms
Speed: 1.8ms preprocess, 38.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 35.0ms
Speed: 1.8ms preprocess, 35.0ms inference, 0.2ms postprocess per image at shape (1, 3, 640, 480)

0: 640x480 1 FACE, 34.5ms
Speed: 1.6ms preprocess, 34.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 480)

0: 640x